In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Feb  2 05:57:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

### Setup and Load dataset

In [ ]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np

In [ ]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget  -b  https://files.grouplens.org/datasets/movielens/ml-25m.zip

Continuing in background, pid 888.
Output will be written to ‘wget-log’.


In [ ]:
!wget  -b  https://files.grouplens.org/datasets/movielens/ml-1m.zip

Continuing in background, pid 890.
Output will be written to ‘wget-log.1’.


In [ ]:
#!gdown https://drive.google.com/uc?id=1iyh-mmcyBaMxZAk_Ey4ncW0mSPNKYcCn

### Setup

In [ ]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np

In [ ]:
!unzip -o "ml-1m.zip"  -d  "/content"

Archive:  ml-1m.zip
   creating: /content/ml-1m/
  inflating: /content/ml-1m/movies.dat  
  inflating: /content/ml-1m/ratings.dat  
  inflating: /content/ml-1m/README   
  inflating: /content/ml-1m/users.dat  


In [ ]:
data = pd.read_csv('ml-1m/ratings.dat', sep='::', header=0, skipinitialspace=True)
data.dropna(inplace=True)


/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
movie_10k_df = data

In [ ]:
movie_10k_df.columns =[ 'userID', 'itemID', 'rating','timestamp']

In [ ]:
# movie_10k_df.rename(columns = {'userId':'userID'}, inplace = True)
# movie_10k_df.rename(columns = {'movieId':'itemID'}, inplace = True)

# Apache PySpark ALS

In [ ]:
!pip install pyspark
!pip install recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.0/339.0 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.2/355.2 KB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 33.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.1/310.1 KB 33.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 KB 9.1 MB/s eta 0

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.ml.tuning import CrossValidator
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import FloatType, IntegerType, LongType

from recommenders.utils.spark_utils import start_or_get_spark
from recommenders.evaluation.spark_evaluation import SparkRankingEvaluation, SparkRatingEvaluation
from recommenders.tuning.parameter_sweep import generate_param_grid
from recommenders.datasets.spark_splitters import spark_random_split


## Prepare data for apache spark

In [ ]:
schema = StructType(
    (
        StructField('userID', IntegerType()),
        StructField('itemID', IntegerType()),
        StructField('rating', FloatType()),
        StructField('timestamp', LongType()),
    )
)

In [ ]:
spark = SparkSession.builder.appName("recommendation").getOrCreate()

In [ ]:
spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", "false")

In [ ]:
sparkDF=spark.createDataFrame(movie_10k_df) 

In [ ]:
sp_train, sp_test = spark_random_split(sparkDF, ratio=0.75, seed=42)


## Run ALS

In [ ]:
als = ALS(
    maxIter=15, 
    rank=10,
    regParam=0.05, 
    userCol='userID', 
    itemCol='itemID', 
    ratingCol='rating', 
    coldStartStrategy="drop"
)

model = als.fit(sp_train)

In [ ]:
dfs_pred = model.transform(sp_test).drop('rating')


In [ ]:
users = sp_train.select('userID').distinct()
items = sp_train.select('itemID').distinct()
user_item = users.crossJoin(items)
dfs_pred = model.transform(user_item)

# Remove seen items.
dfs_pred_exclude_train = dfs_pred.alias("pred").join(
    sp_train.alias("train"),
    (dfs_pred['userID'] == sp_train['userID']) & (dfs_pred['itemID'] == sp_train['itemID']),
    how='outer'
)

dfs_pred_final = dfs_pred_exclude_train.filter(dfs_pred_exclude_train["train.Rating"].isNull()) \
    .select('pred.' + 'userID', 'pred.' + 'itemID', 'pred.' + "prediction")

In [ ]:
dfs_pred_final.show()


+------+------+----------+
|userID|itemID|prediction|
+------+------+----------+
|     1|     3|  3.002062|
|     1|    17| 3.9811285|
|     1|    20| 2.8701625|
|     1|    25| 3.0605898|
|     1|    27| 3.1844556|
|     1|    31| 3.2272692|
|     1|    37|  3.578013|
|     1|    43| 3.5962596|
|     1|    49| 3.9482927|
|     1|    52|  3.215296|
|     1|    53| 3.9522793|
|     1|    60| 3.7902617|
|     1|    66| 1.5111282|
|     1|    80| 2.2604136|
|     1|    84|  1.396312|
|     1|    88|  2.606292|
|     1|    94| 3.7063465|
|     1|    96|  2.785767|
|     1|   107| 2.9844754|
|     1|   113| 3.6789162|
+------+------+----------+
only showing top 20 rows



## evaluate results

In [ ]:
evaluations = SparkRankingEvaluation(
    sp_test, 
    dfs_pred_final,
    col_user='userID',
    col_item='itemID',
    col_rating='rating',
    col_prediction='prediction',
    k=10
)

print(
    "Precision@k = {}".format(evaluations.precision_at_k()),
    "Recall@k = {}".format(evaluations.recall_at_k()),
    "NDCG@k = {}".format(evaluations.ndcg_at_k()),
    "Mean average precision = {}".format(evaluations.map_at_k()),
    sep="\n"
)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Precision@k = 0.03563503891372745
Recall@k = 0.010139463061880346
NDCG@k = 0.029889414095291507
Mean average precision = 0.0023998699443582754
